In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Clean data

## Gift

In [3]:
def gift_clean(file):
    gift_data = pd.read_excel(file, parse_dates = True)
    
    gift_data.columns = gift_data.columns.str.lower().str.strip().str.replace(' ', '_')
    gift_data = gift_data.rename(columns={'gf_apls_1_01_apatrcat_2_01_description':'business_unit'})
    
    
    gift_data['first'] = (gift_data['date'] == gift_data['first_gift_date']).astype(int)
    
    gift_cols = ['date','amount', 'payment_method', 'gift_type', 'fund',
             'key_indicator', 'first', 'business_unit','donor_id']
    gift_data = gift_data[gift_cols]
    
    gift_data = gift_data.dropna()
    
    gift_data = gift_data.merge(pd.get_dummies(gift_data['key_indicator']).rename(columns={'O':'org','I':'individual'})
                            ,left_index=True, right_index=True)
    
    #payment method
    gift_data['payment_method'] = gift_data['payment_method'].where(gift_data['payment_method'] != 'Venmo', other = 'Other',)
    
    gift_data = gift_data.merge(pd.get_dummies(gift_data['payment_method'], 
                              prefix='pm').drop('pm_Other', axis=1)
                ,left_index=True, right_index=True)
    
    #gift type
    gift_data = gift_data.merge(pd.get_dummies(gift_data['gift_type'], 
                              prefix='gt'),
                left_index=True, right_index=True)
    
    #fund
    gift_data['fund'] = gift_data['fund'].map({'Unrestricted':'unrestricted', 
                                           'MN Central Kitchen':'mn_central_kitchen', 
                                           'Restricted-Other':'other'}, 
                                       na_action='other')
    gift_data = gift_data.merge(pd.get_dummies(gift_data['fund'], 
                              prefix='f'),
                left_index=True, right_index=True)
    
    
    gift_data = gift_data.set_index('date')
    categorical = ['payment_method','gift_type','fund','key_indicator','business_unit']
    
    return gift_data.drop(categorical,axis = 1)

## Media

In [12]:
def media_clean(file):
    media_data = pd.read_excel(file, parse_dates = True)
    media_data.columns = media_data.columns.str.lower().str.strip().str.replace(' ', '_')
    media_data = media_data[ media_data['country']=='United States']
    
    #drop duplicate media
    media_data = media_data.drop_duplicates(subset=['date','source','headline'])
    
    media_data['state'] = media_data['state'].fillna('Unknow state')
    media_data['city'] = media_data['city'].fillna('Unknow city')
    
    media_data['headline'] = media_data['headline'].str.lower()
    
    mask = (
            (media_data['headline'].str.contains('second harvest heartland')) |
            (media_data['headline'].str.contains('second harvest')) |
            (media_data['headline'].str.contains('harvest heartland'))  |
            (media_data['headline'].str.contains('heartland')) 
            )
    media_data['in_headline'] = mask
    
    
    cols_2_keep = [ 'in_headline','date','reach', 'desktop_reach', 'mobile_reach', 
                'twitter_social_echo', 'facebook_social_echo', 'reddit_social_echo',
                'sentiment', 'modified_input_media','state']
    media_data = media_data[cols_2_keep]
    
    media_data = media_data.merge(pd.get_dummies(media_data['modified_input_media']),left_index=True, right_index=True)
    
    #sentiment
    media_data = media_data.merge(pd.get_dummies(media_data['sentiment']),left_index=True, right_index=True)
    
    media_data['date'] = pd.to_datetime(media_data['date'])
    
    categoriacl_m = ['sentiment','modified_input_media','state']
    media_data = media_data.drop(categoriacl_m, axis=1)
    media_data = media_data.set_index('date')
    
    return media_data
    

## Campaign

In [5]:
def campaign_clean(file):
    campaign_data = pd.read_excel(file, parse_dates = True)
    
    campaign_data.columns = campaign_data.columns.str.lower().str.strip().str.replace(' ', '_')
    campaign_data['date_text'] = pd.to_datetime(campaign_data['date_text'] )
    campaign_data = campaign_data[(campaign_data['date_text'] <'2022-04-30') 
                                  & (campaign_data['date_text'] >= '2020-05-01')]
    campaign_data = campaign_data.set_index('date_text')
    campaign_data = campaign_data.sort_index()
    
    return campaign_data
    

# Aggregate

In [6]:
def pct(x):
    return x.sum()/x.size

In [7]:
def gift_agg(gift_data,y):
    
    
    sum_g_amt = gift_data.groupby(pd.Grouper(freq="1D")).agg({'amount':['sum','size']}).drop('2022-04-30',errors='ignore')
   
    gift_pct_columns = ['first', 'individual', 'org', 'pm_Business Check', 'pm_Cash',
       'pm_Credit Card', 'pm_Personal Check', 'gt_Cash', 'gt_Pay-Cash',
       'gt_Recurring Gift', 'gt_Stock/Property', 'f_mn_central_kitchen',
       'f_other', 'f_unrestricted']
    
    sum_g_pct = gift_data.groupby(pd.Grouper(freq="1D"))[gift_pct_columns].apply(sum)
    
    
    return (sum_g_amt['amount'][y].to_frame(), sum_g_amt)
    

In [8]:
def media_agg(media_data, wp_d, wa_d, wv_d, tv_d, print_d, radio_d):

    
    media_data = media_data.sort_index()
    
    media_lags = {'Web/Print':wp_d,
                'TV':tv_d,
                'Web-Article':wa_d,
                'Radio':radio_d,
                'Print':print_d,
                'Web-Video':wv_d}
    
    media_rolling = pd.DataFrame()
    for col, lag in media_lags.items():
        mt_day = media_data[media_data[col]==1].resample('D').agg(sum).fillna(0)
        mt_rolling = mt_day.rolling(lag).agg(sum).fillna(0)
        if media_rolling.empty:
            media_rolling = mt_rolling
        else:
            media_rolling = media_rolling.add(mt_rolling, fill_value=0)
            
    media_rolling['holiday'] = media_rolling.index.month.isin([11, 12, 1]).astype(int)

    return media_rolling
    
    

# Import data

In [9]:
gift_data = gift_clean('/Users/hao/Desktop/MSBA 6130 (R)/live case/Outbox/Gift Data File - SHH.xlsx')
media_data = media_clean('/Users/hao/Desktop/MSBA 6130 (R)/live case/Outbox/SHH-Source_categorized_AA.xlsx')
campaign_data = campaign_clean('/Users/hao/Desktop/MSBA 6130 (R)/live case/Outbox/Campaign_date_weights.xlsx')

In [13]:
media_data = media_clean('/Users/hao/Desktop/MSBA 6130 (R)/live case/Outbox/SHH-Source_categorized_AA.xlsx')

,in_headline,reach,desktop_reach,mobile_reach,twitter_social_echo,facebook_social_echo,reddit_social_echo,Print,Radio,TV,Web-Article,Web-Video,Web/Print,Negative,Neutral,Not Rated,Positive
date,,,,,,,,,,,,,,,,,
2022-04-27 09:16:00,True,1178357,381530,796827,1,0,0,0,0,0,1,0,0,0,1,0,0
2022-04-27 06:58:00,True,1178357,381530,796827,0,0,0,0,0,0,0,1,0,1,0,0,0
2022-04-22 10:41:00,True,1178357,381530,796827,14,3,0,0,0,0,1,0,0,0,0,0,1
2022-04-12 18:39:00,True,265,265,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2022-04-12 14:59:00,True,117,117,0,0,0,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-01 12:58:00,True,7720,5116,2604,2,0,0,0,0,0,1,0,0,0,1,0,0
2020-05-27 15:01:00,True,6,2,4,0,0,0,0,0,0,0,0,1,0,0,0,1
2020-05-08 18:55:00,True,1497585,394271,1103314,0,97,0,0,0,0,0,1,0,0,1,0,0


# Agg

In [14]:
campaign = campaign_data.rolling('20D').apply(pct)

In [15]:
media = media_agg(media_data.copy(),'16D', '16D', '1D', '2D', '2D', '29D')
media

,in_headline,reach,desktop_reach,mobile_reach,twitter_social_echo,facebook_social_echo,reddit_social_echo,Print,Radio,TV,Web-Article,Web-Video,Web/Print,Negative,Neutral,Not Rated,Positive,holiday
date,,,,,,,,,,,,,,,,,,
2020-05-01,0.0,372519.0,43576.0,81767.0,3.0,7.0,0.0,0.0,1.0,5.0,1.0,0.0,5.0,0.0,8.0,0.0,4.0,0
2020-05-02,0.0,610127.0,62607.0,107196.0,15.0,541.0,0.0,0.0,2.0,9.0,1.0,0.0,6.0,0.0,9.0,0.0,9.0,0
2020-05-03,1.0,4030310.0,1120238.0,2432944.0,47.0,875.0,0.0,0.0,2.0,8.0,1.0,0.0,7.0,0.0,8.0,0.0,10.0,0
2020-05-04,1.0,6868422.0,2190746.0,4048996.0,54.0,919.0,0.0,0.0,2.0,11.0,1.0,0.0,10.0,0.0,12.0,0.0,12.0,0
2020-05-05,1.0,6960557.0,2247513.0,4120377.0,68.0,1203.0,1.0,0.0,2.0,13.0,1.0,0.0,12.0,1.0,15.0,0.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,66.0,81313363.0,28592396.0,51288393.0,49.0,926.0,0.0,0.0,5.0,3.0,68.0,0.0,11.0,0.0,81.0,0.0,6.0,0
2022-04-26,66.0,82677523.0,28593296.0,51289827.0,49.0,926.0,0.0,2.0,5.0,20.0,68.0,0.0,11.0,0.0,100.0,0.0,6.0,0
2022-04-27,10.0,86724816.0,29027614.0,52520331.0,51.0,926.0,0.0,0.0,7.0,62.0,13.0,1.0,10.0,1.0,89.0,0.0,3.0,0


In [16]:
gift_y, gift_features = gift_agg(gift_data.copy(),'sum')

In [85]:
media.columns

Index(['in_headline', 'reach', 'desktop_reach', 'mobile_reach',
       'twitter_social_echo', 'facebook_social_echo', 'reddit_social_echo',
       'Print', 'Radio', 'TV', 'Web-Article', 'Web-Video', 'Web/Print',
       'Negative', 'Neutral', 'Not Rated', 'Positive', 'holiday'],
      dtype='object')

In [17]:
#merge campaign
features = media.merge(campaign, how = 'outer',left_index = True,right_index = True)
features = features.fillna(0)

In [45]:

features = features.merge(gift_features,left_index = True,right_index = True )

In [18]:
result = features.merge(gift_y, left_index = True,right_index = True)
result

,in_headline,reach,desktop_reach,mobile_reach,twitter_social_echo,facebook_social_echo,reddit_social_echo,Print,Radio,TV,...,Positive,holiday,acquisition_mail,cultivation_mail_appeal,e-appeals,newspaper_inserts,other_mailings,print_newsletter,shh_events,sum
2020-05-01,0.0,372519.0,43576.0,81767.0,3.0,7.0,0.0,0.0,1.0,5.0,...,4.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,94736.79
2020-05-02,0.0,610127.0,62607.0,107196.0,15.0,541.0,0.0,0.0,2.0,9.0,...,9.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,30441.00
2020-05-03,1.0,4030310.0,1120238.0,2432944.0,47.0,875.0,0.0,0.0,2.0,8.0,...,10.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,42980.00
2020-05-04,1.0,6868422.0,2190746.0,4048996.0,54.0,919.0,0.0,0.0,2.0,11.0,...,12.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,269304.34
2020-05-05,1.0,6960557.0,2247513.0,4120377.0,68.0,1203.0,1.0,0.0,2.0,13.0,...,12.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,149346.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,66.0,81313363.0,28592396.0,51288393.0,49.0,926.0,0.0,0.0,5.0,3.0,...,6.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,55522.03
2022-04-26,66.0,82677523.0,28593296.0,51289827.0,49.0,926.0,0.0,2.0,5.0,20.0,...,6.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,57255.90
2022-04-27,10.0,86724816.0,29027614.0,52520331.0,51.0,926.0,0.0,0.0,7.0,62.0,...,3.0,0,0.0,0.833333,0.0,0.0,0.0,0.0,0.166667,412029.38
2022-04-28,0.0,9637247.0,2051725.0,5548404.0,14.0,130.0,0.0,0.0,0.0,47.0,...,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,49836.37


In [19]:
result.columns = result.columns.str.lower().str.strip().str.replace(' ', '_')
result.columns = result.columns.str.lower().str.strip().str.replace('-', '_')
result.columns = result.columns.str.lower().str.strip().str.replace('/', '_')

In [20]:
result.to_excel('donation_amt_media_variables.xlsx', index=False)

# Linear

In [17]:
x = result.loc[:, result.columns != 'sum']

In [24]:
y = result['sum']

In [87]:
x = features
y = gift_y
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
model.summary()

/Users/hao/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    sum   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     6.764
Date:                Tue, 09 Aug 2022   Prob (F-statistic):           3.01e-19
Time:                        16:54:08   Log-Likelihood:                -9833.7
No. Observations:                 729   AIC:                         1.972e+04
Df Residuals:                     705   BIC:                         1.983e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    3.602e+04   1.25e+04      2.871      0.004    1.14e+04    6.07e+04
in_headline             -9.468e-05   5.13e-05     -1.846      0.065      -0.000    6.01e-06
reach                       0.0019      0.010      0.198      0.843      -0.017       0.021
desktop_reach              -0.0069      0.010     -0.662      0.508      -0.028       0.014
mobile_reach                0.0006      0.010      0.062      0.951      -0.018       0.019
twitter_social_echo       341.8943     84.655      4.039      0.000     175.687     508.101
facebook_social_echo        1.9192      1.528      1.256      0.210      -1.081       4.919
reddit_social_echo      -9249.1915   2192.404     -4.219      0.000   -1.36e+04   -4944.769
Print                   -1.146e+04   4.19e+04     -0.274      0.785   -9.37e+04    7.08e+04
Radio                   -9215.9472   7936.593     -1.161      0.246   -2.48e+04    6366.240
TV                      -9964.4573   7951.945     -1.253      0.211   -2.56e+04    5647.872
Web-Article             -9647.6770   7983.003     -1.209      0.227   -2.53e+04    6025.628
Web-Video                5.039e+04   3.97e+04      1.268      0.205   -2.76e+04    1.28e+05
Web/Print               -8837.3581   7941.391     -1.113      0.266   -2.44e+04    6754.249
Negative                 9873.8539   8492.040      1.163      0.245   -6798.861    2.65e+04
Neutral                  1.023e+04   7969.628      1.283      0.200   -5420.222    2.59e+04
Not Rated               -3.155e+04   4.35e+04     -0.725      0.469   -1.17e+05    5.39e+04
Positive                 1.272e+04   7974.909      1.595      0.111   -2938.643    2.84e+04
holiday                  4.556e+04   1.94e+04      2.349      0.019    7484.957    8.36e+04
acquisition_mail         -4.05e+05   2.71e+05     -1.497      0.135   -9.36e+05    1.26e+05
cultivation_mail_appeal -3.138e+04   2.39e+04     -1.312      0.190   -7.83e+04    1.56e+04
e-appeals                1.194e+05    4.5e+04      2.653      0.008     3.1e+04    2.08e+05
newspaper_inserts       -9890.6692   7.96e+04     -0.124      0.901   -1.66e+05    1.46e+05
other_mailings           8.911e+04   1.09e+05      0.817      0.414   -1.25e+05    3.03e+05
print_newsletter        -1.307e+05   1.74e+05     -0.753      0.452   -4.72e+05     2.1e+05
shh_events               2.353e+05   1.51e+05      1.555      0.120   -6.18e+04    5.32e+05
==============================================================================
Omnibus:                      800.249   Durbin-Watson:                   1.844
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70812.312
Skew:                           5.086   Prob(JB):                         0.00
Kurtosis:                      50.200   Cond. No.                     1.34e+16
====================